In [62]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [63]:
# Kerasに付属の手書き数字画像データをダウンロード
np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()

In [64]:
# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2)

# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
X_train = X_train.reshape(-1,784)
X_validation = X_validation.reshape(-1,784)
X_test = X_test.reshape(-1,784)

#正規化
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

In [65]:
# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

In [66]:
# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.01

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
model.add(Dense(n_hidden,activation='relu',input_shape=(n_features,)))
model.add(Dense(n_hidden,activation='relu'))
model.add(Dense(n_hidden,activation='relu'))

# 出力層を追加
model.add(Dense(10,activation='softmax'))

In [67]:
# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['mae', 'accuracy'])

# Early stoppingを適用してフィッティング
log = model.fit(X_train, y_train, epochs=3000, batch_size=100, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     min_delta=0, patience=10, 
                                                         verbose=1)],
                validation_data=(X_validation, y_validation))

Epoch 1/3000
480/480 [==============================] - 1s 1ms/step - loss: 0.2761 - mae: 0.0241 - accuracy: 0.9156 - val_loss: 0.1598 - val_mae: 0.0141 - val_accuracy: 0.9521
Epoch 2/3000
480/480 [==============================] - 1s 1ms/step - loss: 0.1555 - mae: 0.0130 - accuracy: 0.9550 - val_loss: 0.1406 - val_mae: 0.0121 - val_accuracy: 0.9597
Epoch 3/3000
480/480 [==============================] - 1s 1ms/step - loss: 0.1234 - mae: 0.0102 - accuracy: 0.9654 - val_loss: 0.1398 - val_mae: 0.0099 - val_accuracy: 0.9603
Epoch 4/3000
480/480 [==============================] - 1s 1ms/step - loss: 0.1117 - mae: 0.0092 - accuracy: 0.9676 - val_loss: 0.1371 - val_mae: 0.0110 - val_accuracy: 0.9620
Epoch 5/3000
480/480 [==============================] - 1s 1ms/step - loss: 0.1003 - mae: 0.0079 - accuracy: 0.9720 - val_loss: 0.1375 - val_mae: 0.0103 - val_accuracy: 0.9647
Epoch 6/3000
480/480 [==============================] - 1s 1ms/step - loss: 0.0967 - mae: 0.0075 - accuracy: 0.9731 - va

In [68]:
# Test dataで予測を実行。
pred_test = np.argmax(model.predict(X_test), axis=1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

9649/10000 correct (96.490%)
